<a href="https://colab.research.google.com/github/ajinkyagh/Python_Practice/blob/main/Practice_Python_For_DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Day 1 — Ingest and audit raw data
Goal
Load the raw CSVs and do a quick health check so we know what we are dealing with before cleaning.

Your tasks
Create a notebook or script called 01_ingest_and_audit.py or .ipynb

Load these five files into DataFrames:

customers.csv

products.csv

orders.csv

order_items.csv

returns.csv

Use parse_dates for any date columns.

For each DataFrame, report:

shape and first 5 rows

info() and basic describe() for numeric columns

missing values per column

duplicate rows count

obvious dtype issues to fix later

Integrity checks

Every order_items.order_id exists in orders.order_id

Every orders.customer_id exists in customers.customer_id

Every order_items.product_id exists in products.product_id

Optional: if returns exists, every returns.order_id exists in orders.order_id

Write a very short audit summary to reports/day01_audit.md:

key counts

columns with missing values

any suspicious dtypes

integrity check results

Hints
Use pd.read_csv(..., parse_dates=[...]) for date columns

df.isna().sum() for missing counts

df.duplicated().sum() for duplicates

Use set difference to check keys across tables

In [5]:
from pathlib import Path
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

# Load
customers = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/customers.csv', parse_dates=['signup_date'])
products  = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/products.csv')
orders    = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/orders.csv',
                        parse_dates=['order_date'])
order_items = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/order_items.csv')
returns   = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/returns.csv')

tables = {
    "customers": customers,
    "products": products,
    "orders": orders,
    "order_items": order_items,
    "returns": returns,
}

def audit_df(name, df):
    print(f"\n=== {name.upper()} ===")
    print("shape:", df.shape)
    display(df.head(5))
    print(df.info())
    print("missing per column:\n", df.isna().sum())
    print("duplicate rows:", df.duplicated().sum())
    num_cols = df.select_dtypes(include="number").columns
    if len(num_cols):
        display(df[num_cols].describe())

for name, df in tables.items():
    audit_df(name, df)

# Integrity checks (sets -> counts + peek)
bad_oi_orders = set(order_items["order_id"]) - set(orders["order_id"])
bad_orders_customers = set(orders["customer_id"]) - set(customers["customer_id"])
bad_oi_products = set(order_items["product_id"]) - set(products["product_id"])
bad_returns_orders = set(returns["order_id"]) - set(orders["order_id"])

print("\n=== Integrity checks ===")
print(f"order_items.order_id not in orders: {len(bad_oi_orders)}; sample: {list(bad_oi_orders)[:5]}")
print(f"orders.customer_id not in customers: {len(bad_orders_customers)}; sample: {list(bad_orders_customers)[:5]}")
print(f"order_items.product_id not in products: {len(bad_oi_products)}; sample: {list(bad_oi_products)[:5]}")
print(f"returns.order_id not in orders: {len(bad_returns_orders)}; sample: {list(bad_returns_orders)[:5]}")

# Write a short markdown audit
reports = Path("/content/gdrive/My Drive/Colab Notebooks/dataset/reports")
reports.mkdir(parents=True, exist_ok=True)

def mk(title, value): return f"- {title}: {value}\n"

with open(reports / "day01_audit.md", "w") as f:
    f.write("# Day 1 Audit\n\n")
    for name, df in tables.items():
        f.write(f"## {name}\n")
        f.write(mk("rows", df.shape[0]))
        f.write(mk("cols", df.shape[1]))
        na = df.isna().sum()
        na = na[na > 0]
        f.write(mk("columns with missing", na.to_dict()))
        f.write(mk("duplicate rows", int(df.duplicated().sum())))
        # quick dtype note
        f.write(mk("dtypes", df.dtypes.astype(str).to_dict()))
        f.write("\n")
    f.write("## Integrity checks\n")
    f.write(mk("order_items.order_id not in orders", len(bad_oi_orders)))
    f.write(mk("orders.customer_id not in customers", len(bad_orders_customers)))
    f.write(mk("order_items.product_id not in products", len(bad_oi_products)))
    f.write(mk("returns.order_id not in orders", len(bad_returns_orders)))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

=== CUSTOMERS ===
shape: (10, 4)


,customer_id,name,signup_date,email
0,1,Alice,2021-01-01,user1@example.com
1,2,Bob,2021-04-01,user2@example.com
2,3,Charlie,2021-06-30,user3@example.com
3,4,David,2021-09-28,NaN
4,5,Eva,2021-12-27,user5@example.com


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  10 non-null     int64         
 1   name         10 non-null     object        
 2   signup_date  10 non-null     datetime64[ns]
 3   email        9 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 452.0+ bytes
None
missing per column:
 customer_id    0
name           0
signup_date    0
email          1
dtype: int64
duplicate rows: 0


,customer_id
count,10.00000
mean,5.50000
std,3.02765
min,1.00000
25%,3.25000
50%,5.50000
75%,7.75000
max,10.00000



=== PRODUCTS ===
shape: (11, 4)


...


=== RETURNS ===
shape: (5, 4)


,return_id,order_id,return_date,reason
0,1,1002,2022-06-08,Damaged
1,2,7777,2022-07-06,Damaged
2,3,1013,2022-08-14,Damaged
3,4,1006,2022-06-27,Other
4,5,1009,2022-07-15,Damaged


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   return_id    5 non-null      int64 
 1   order_id     5 non-null      int64 
 2   return_date  5 non-null      object
 3   reason       5 non-null      object
dtypes: int64(2), object(2)
memory usage: 292.0+ bytes
None
missing per column:
 return_id      0
order_id       0
return_date    0
reason         0
dtype: int64
duplicate rows: 0


,return_id,order_id
count,5.000000,5.000000
mean,3.000000,2361.400000
std,1.581139,3027.415119
min,1.000000,1002.000000
25%,2.000000,1006.000000
50%,3.000000,1009.000000
75%,4.000000,1013.000000
max,5.000000,7777.000000



=== Integrity checks ===
order_items.order_id not in orders: 1; sample: [8888]
orders.customer_id not in customers: 1; sample: [999]
order_items.product_id not in products: 1; sample: [999]
returns.order_id not in orders: 1; sample: [7777]


Day 2 — Quick targeted cleaning
Goal
Fix obvious data-quality issues you found yesterday so joins won’t break.

Tasks (short!)
De-dup products

Drop exact duplicate rows.

Fix foreign keys

Remove orders with customer_id not in customers.

Remove order_items with order_id not in orders.

Remove order_items with product_id not in products.

Handle missing email

For customers.email, fill missing with a placeholder like unknown_{customer_id}@example.com.

Hints
df.drop_duplicates(inplace=True)

Anti-join filter with isin:

In [6]:
#Drop Duplicates
products = products.drop_duplicates()

#Fix Foreign keys
orders = orders[orders["customer_id"].isin(customers["customer_id"])]
order_items = order_items[order_items["order_id"].isin(orders["order_id"])]
order_items = order_items[order_items["product_id"].isin(products["product_id"])]
returns = returns[returns["order_id"].isin(orders["order_id"])]


#Missing emails
customers["email"] = customers["email"].fillna(
    customers["customer_id"].astype(str).radd("unknown_").add("@example.com")
)


# Save cleaned versions
customers.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/clean_customers.csv', index=False)
products.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/clean_products.csv', index=False)
orders.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/clean_orders.csv', index=False)
order_items.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/clean_order_items.csv', index=False)
returns.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/clean_returns.csv', index=False)

Day 3 — Type fixes + simple outlier checks
Goal

Make sure columns have the right dtypes and quickly flag obvious outliers so later joins/aggregations behave.

Tasks (quick)

Type audit & fixes

Ensure these columns have correct dtypes:

customers.signup_date → datetime64[ns]

orders.order_date → datetime64[ns]

products.price → numeric (coerce errors to NaN)

order_items.quantity → integer (coerce then fill bads with 0 or 1)

After coercion, report how many values became NaN (that’s your “bad type” count).

Outlier checks (simple)

For products.price: flag rows where price ≤ 0 or price > 99th percentile.

For order_items.quantity: flag rows where qty ≤ 0 or qty > 95th percentile.

Just print counts and show the top 5 offending rows for each.

Save typed versions

Save as:

typed_customers.csv, typed_products.csv, typed_orders.csv, typed_order_items.csv, typed_returns.csv

In [11]:
customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce")
orders["order_date"] = pd.to_datetime(orders["order_date"], errors="coerce")

products["price"] = pd.to_numeric(products["price"], errors="coerce")

order_items["quantity"] = pd.to_numeric(order_items["quantity"], errors="coerce")
bad_quantity = order_items["quantity"].isna()

/tmp/ipython-input-1405645508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders["order_date"] = pd.to_datetime(orders["order_date"], errors="coerce")
/tmp/ipython-input-1405645508.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products["price"] = pd.to_numeric(products["price"], errors="coerce")
